# End-to-end OptiMUS pipeline

This notebook runs the full flow: **problem description + data** → **OptiMUS inputs** → **OptiMUS run** → **solution**.

1. **Setup** – paths and imports  
2. **Inputs** – paths to data and description  
3. **Step 1** – Generate OptiMUS inputs (model_input/desc.txt, model_input/params.json) from description + data  
4. **Step 2** – Run OptiMUS pipeline (extract objective, constraints, generate code, solve)  
5. **Step 3** – Load and display the solution

In [ ]:
# Setup: project root and imports
import os
import sys
import subprocess
from pathlib import Path

# Find project root (directory that contains data_to_optimax.py), regardless of cwd
def _find_project_root() -> Path:
    start = Path.cwd().resolve()
    for d in [start] + list(start.parents):
        if (d / "data_to_optimax.py").exists():
            return d
    return start  # fallback

PROJECT_ROOT = _find_project_root()
os.chdir(PROJECT_ROOT)
sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")

: 

## Inputs

Set paths to your **data file(s)** (CSV/Excel) and **problem description** (text or path to `.txt`). Use a single path or a list of paths for multiple datasets (e.g. `DATA_PATH = ["inventory.csv", "stores.csv"]`). Output directory is where OptiMUS input files will be written.

In [12]:
# Paths (relative to project root after Setup cell)
# Example: inventory problem in scripts/. Use one path or a list for multiple datasets.
DATA_PATH = "current_query/raw_input/inventory_monitoring.csv"
DESCRIPTION_PATH = "current_query/raw_input/desc.txt"
OUTPUT_DIR = "current_query"
MODEL = "gpt-4o-mini"

# Optional: use --no-llm for no API calls, or --simple for one param per column
USE_EXPERT = True

## Step 1: Generate OptiMUS inputs

Convert **description + data** into `model_input/desc.txt` and `model_input/params.json`. Uses expert (LLM) extraction by default to map columns to parameters.

In [13]:
import importlib.util
_spec = importlib.util.spec_from_file_location(
    "data_to_optimax",
    Path(PROJECT_ROOT) / "data_to_optimax.py",
)
_data_to_optimax = importlib.util.module_from_spec(_spec)
_spec.loader.exec_module(_data_to_optimax)

_data_to_optimax.run(
    DATA_PATH,
    DESCRIPTION_PATH,
    output_dir=OUTPUT_DIR,
    model=MODEL,
    use_expert=USE_EXPERT,
    no_llm=False,
)
print("OptiMUS inputs written to", OUTPUT_DIR)

Loaded 31 rows, 9 columns from current_query/raw_input/inventory_monitoring.csv (dataset: inventory_monitoring)
Extracting parameters with expert (LLM) reasoning over description and dataset(s)...
Wrote raw_input/ and model_input/ under current_query/
  model_input/desc.txt, model_input/params.json (8 parameters)

Next: run OptiMUS with:
  python optimus.py
OptiMUS inputs written to current_query


## Step 2: Run OptiMUS pipeline

Run the full OptiMUS pipeline on the generated folder: extract objective and constraints, formulate them, generate Gurobi code, execute and debug until a solution is found.

In [ ]:
# Before running, make sure youhave your Gurobi license file in your home directory !
from optimus import run_pipeline

state = run_pipeline(
    problem_dir=OUTPUT_DIR,
    model=MODEL,
    error_correction=True,
)
print("Pipeline finished. State keys:", list(state.keys()))

{'description': '"The goal is to reduce stockouts on high-demand products, minimize holding and markdown costs, and avoid costly emergency shipments."', 'formulation': None, 'code': None}
_________________________ get_constraints _________________________
[{'description': 'Current stock levels at each store must be non-negative', 'formulation': None, 'code': None}, {'description': "Total stock allocated to each product across all stores must not exceed the product's supply capacity at the warehouse", 'formulation': None, 'code': None}, {'description': 'Stock levels after each reorder must satisfy the reorder point for each product', 'formulation': None, 'code': None}, {'description': 'Number of product reorders must be feasible within supplier lead times', 'formulation': None, 'code': None}, {'description': 'Total inventory across all warehouses must not exceed their capacity', 'formulation': None, 'code': None}, {'description': 'Frequency of stockouts for each product must be minimize

## Step 3: Load and display the solution

Read the solver output and optional solution file from `optimus_output/`.

In [9]:
output_dir = Path(PROJECT_ROOT) / OUTPUT_DIR / "optimus_output"

# Solver stdout (often contains "Optimal Objective Value: ...")
code_output = output_dir / "code_output.txt"
if code_output.exists():
    print("--- code_output.txt ---")
    print(code_output.read_text())
else:
    print("code_output.txt not found")

# Written optimal value (if status was OPTIMAL)
solution_file = output_dir / "output_solution.txt"
if solution_file.exists():
    print("\n--- output_solution.txt ---")
    print(solution_file.read_text())

--- code_output.txt ---
Execution failed:
Traceback (most recent call last):
  File "/Users/hindy/Desktop/OptiMUS/current_query/optimus_output/code_2.py", line 4, in <module>
    from gurobipy import Model, GRB, quicksum
ModuleNotFoundError: No module named 'gurobipy'

